In [2]:
using Random
using DataStructures

In [3]:
# returns dup char --> num instances dictionary, num of duplicate chars  
##### returns 2 x N array (N = num dup char) of duplicate char & num instances, num of duplicate chars

function find_dups_in_str(str::String)
    alphabet = Set{Char}() 
    duplicates = OrderedDict{Char, Int}()  # duplicate char --> num instances 

    for char in str   
        # print(char)
        if char in alphabet 
            if char in keys(duplicates)
                duplicates[char] += 1
            else 
                duplicates[char] = 2 
            end 
        else 
            push!(alphabet, char)
        end 
    end 

    return duplicates, length(duplicates)
end 

find_dups_in_str (generic function with 1 method)

In [12]:
# mapping S --> T, generates r random maps 

function randommap(S::String, P::String, r::Int)
    S_dup_to_num_instances, S_num_dups = find_dups_in_str(S)
    P_dup_to_num_instances, P_num_dups = find_dups_in_str(P)
    
    print(S_dup_to_num_instances, "\n")
    # print(T_dup_to_num_instances)

    S_M = Array{Array{Int}}(undef, r)  # does copy elements or point to them? 

    # r RM of src str S are generated and stored in a set S_M 
    for i in 1:r 
    # ONE MAP 
        S_map = Array{Int}(undef, S_num_dups)
        
        idx = 1
        # for each position of the map (each gene w >1 instances)
        for (dup_char, num_instances) in pairs(S_dup_to_num_instances)
            # an integer value is selcted uniformly from the interval [0, occ(α, S)! -1] 
            mapidx_for_dup_char = rand(0:factorial(num_instances)-1)
            
            S_map[idx] = mapidx_for_dup_char
            idx += 1
        end 
            println(S_map)
            push!(S_M, S_map)
    
    end 
        
    # choose an arbitrary map p for target string 
    P_map = Array{Int}(undef, P_num_dups)
        
    idx = 1
    # for each position of the map (each gene w >1 instances)
    for (dup_char, num_instances) in pairs(P_dup_to_num_instances)
        # an integer value is selcted uniformly from the interval [0, occ(α, S)! -1] 
        mapidx_for_dup_char = rand(0:factorial(num_instances)-1)
        
        P_map[idx] = mapidx_for_dup_char
        idx += 1
    end 
    println("\n***\n\n target string map(s)")
    print(P_map, "\n")
        
    

    # lehmer code to convert int n --> nth permutation of n ints in lexicographical order 

   
        
    # intiially the target string is mapped into permutation Pp using an arbitrary map p 
    
    # for each map m in M 
        # distance between permutation Sm and Pp is computed using estimator algo 

    #return shortest map iwth shortest dist between permutations 
end 

src = "aaabbcdeffffgh"
target = "cbaaaa"

randommap(src, target, 10)

OrderedDict('a' => 3, 'b' => 2, 'f' => 4)
[1, 0, 17]
[1, 0, 12]
[2, 0, 8]
[4, 0, 11]
[3, 1, 12]
[3, 1, 22]
[5, 1, 16]
[4, 0, 21]
[3, 1, 3]
[2, 1, 11]

***

 target string map(s)
[21]
